In [1]:
# !pip install lamini
from llama import BasicModelRunner
from llama import LLMEngine

In [2]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import logging
import time
import torch
import transformers
import pandas as pd

In [3]:

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner
from llama import BasicModelRunner

logger = logging.getLogger(__name__)
global_config = None

In [10]:
from saban_utilities import *

In [4]:
# # with open('utilities.py', 'r') as file:
# #     print(file.read())

# ###################
# import os

# # Şu anki çalışma dizinini al
# current_directory = os.getcwd()

# # Çalışma dizinindeki dosyaları ve klasörleri listele
# files_and_folders = os.listdir(current_directory)

# ######################
# from IPython.display import FileLink
# FileLink("/home/jovyan/work/lamini_docs.jsonl")



In [21]:
import pandas as pd
import json

# Parquet dosyasının yolunu belirtin
parquet_file_path = "train-00000-of-00001-6359aa989b671345.parquet"

# Parquet dosyasını oku
df = pd.read_parquet(parquet_file_path)


# Convert the DataFrame to a list of dictionaries (JSON records)
json_records = df.to_dict(orient='records')

# Define the output .jsonl file path
output_jsonl_file = 'lamini_docs.jsonl'

# Write the JSON records to a .jsonl file
with open(output_jsonl_file, 'w') as f:
    for record in json_records:
        f.write(f'{json.dumps(record)}\n')

# Veriyi incele
df.head()

,question,answer
0,How can I evaluate the performance and quality...,There are several metrics that can be used to ...
1,Can I find information about the code's approa...,"Yes, the code includes methods for submitting ..."
2,How does Lamini AI handle requests for generat...,Lamini AI offers features for generating text ...
3,Does the `submit_job()` function expose any ad...,It is unclear which `submit_job()` function is...
4,Does the `add_data()` function support differe...,"No, the `add_data()` function does not support..."


In [6]:
dataset_name = "lamini_docs.jsonl"
dataset_path = f"/content/{dataset_name}"
use_hf = False

In [7]:
dataset_path = "lamini/lamini_docs"
use_hf = True

In [8]:
model_name = "EleutherAI/pythia-70m"

training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

2023-09-18 08:55:51,722 - DEBUG - saban_utilities - Config: datasets.path: lamini/lamini_docs
datasets.use_hf: true
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-70m
verbose: true



tokenize True lamini/lamini_docs


2023-09-18 08:55:53,076 - DEBUG - fsspec.local - open file: /home/sabankara/.cache/huggingface/datasets/lamini___parquet/lamini--lamini_docs-c73aea7bc123faf0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/dataset_info.json
2023-09-18 08:55:53,099 - WARNING - datasets.builder - Found cached dataset parquet (/home/sabankara/.cache/huggingface/datasets/lamini___parquet/lamini--lamini_docs-c73aea7bc123faf0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
2023-09-18 08:55:53,100 - DEBUG - fsspec.local - open file: /home/sabankara/.cache/huggingface/datasets/lamini___parquet/lamini--lamini_docs-c73aea7bc123faf0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/dataset_info.json


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})


In [12]:
#NOTE Load the base model
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [13]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
    print("x")    
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")
    print("z")

2023-09-18 08:56:01,655 - DEBUG - saban_utilities - Select GPU device


x


In [14]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=512, out_features=50304, bias=False)
)

In [15]:
#NOTE Define function to carry out inference
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
    input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
    )

  # Generate
    device = model.device
    generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
    )

    # Decode
    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

    # Strip the prompt
    generated_text_answer = generated_text_with_prompt[0][len(text):]

    return generated_text_answer

In [16]:
#NOTE Try the base model
test_text = test_dataset[0]['question']
print(">>>")
print("Question input (test):", test_text,"\n")
print(">>>>>>")
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}","\n")
print(">>>>>>>>>")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


>>>
Question input (test): Can Lamini generate technical documentation or user manuals for software projects? 

>>>>>>
Correct answer from Lamini docs: Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects. 

>>>>>>>>>
Model's answer: 


I have a question about the following:

How do I get the correct documentation to work?

A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.

A:

You can use the following code to get the correct documentation.

A:

You can use the following


In [17]:
#NOTE Setup training
max_steps = 3
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [18]:
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)
print(">>>")
print(base_model)
print(">>>>>>")
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print(">>>>>>>>>")
print("Flops", model_flops / 1e9, "GFLOPs")

>>>
GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=512, out_features=50304, bias=False)
)
>>>>>>
Memory footprint 0.30687253

In [19]:
#NOTE Train a few steps

trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

training_output = trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   eval_dataset=test_dataset,                                                              │
│   10 )                                                                                           │
│   11                                                                                             │
│ ❱ 12 training_output = trainer.train()                                                           │
│   13                                                                                             │
│                                                                                                  │
│ /home/sabankara/.conda/envs/llama/lib/python3.10/site-packages/transformers/trainer.py:1628 in   │
│ train                                                                                            │
│                                                                                                  │
│   1625 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1626 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1627 │   │   )                                                                                 │
│ ❱ 1628 │   │   return inner_training_loop(                                                       │
│   1629 │   │   │   args=args,                                                                    │
│   1630 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1631 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/sabankara/.conda/envs/llama/lib/python3.10/site-packages/transformers/trainer.py:1866 in   │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1863 │   │   │   │   rng_to_sync = True                                                        │
│   1864 │   │   │                                                                                 │
│   1865 │   │   │   step = -1                                                                     │
│ ❱ 1866 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1867 │   │   │   │   if rng_to_sync:                                                           │
│   1868 │   │   │   │   │   self._load_rng_state(resume_from_checkpoint)                          │
│   1869 │   │   │   │   │   rng_to_sync = False                                                   │
│                                                                                                  │
│ /home/sabankara/.conda/envs/llama/lib/python3.10/site-packages/torch/utils/data/dataloader.py:63 │
│ 4 in __next__                                                                                    │
│                                                                                                  │
│    631 │   │   │   if self._sampler_iter is None:                                                │
│    632 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    633 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  634 │   │   │   data = self._next_data()                                                      │
│    635 │   │   │   self._num_yielded += 1                                                        │
│    636 │   │   │   if self._dataset_kind == _DatasetKind.It